In [1]:
import pandas as pd
import numpy as np

In [8]:
# Load dataframe
accounts_df = pd.read_csv('/home/jovyan/shared/datasets/RetailBanking/accounts_all.csv')

In [34]:
# Temporarily fill nan values in Limit column with 0 to avoid issues with arithmetic aggregations
accounts_df['Limit'] = accounts_df['Limit'].fillna(value=0)
accounts_df.head()

# One hot encode type to make aggregation easier
accounts_df['IsCredit'] = (accounts_df['Type'] == 'credit_card')
accounts_df['IsChecking'] = (accounts_df['Type'] == 'checking')
accounts_df['IsSavings'] = (accounts_df['Type'] == 'savings')
accounts_df
#print('Sanity check: ', accounts_df['Type'].nunique())

,CustomerID,AccountID,Type,Balance,Limit,OpenDate,IsCredit,IsChecking,IsSavings
0,C000001,Aa95695be,credit_card,535.150521,24081.773437,2019-07-02,True,False,False
1,C000002,A04c87035,credit_card,188.568144,8485.566470,2020-09-17,True,False,False
2,C000002,A7464b435,checking,150.321730,0.000000,2020-11-22,False,True,False
3,C000003,A0c7e1cc4,checking,364.620994,0.000000,2024-06-23,False,True,False
4,C000003,Ac8da0e6b,checking,608.607031,0.000000,2024-09-17,False,True,False
...,...,...,...,...,...,...,...,...,...
1638,C000999,A009e5f1d,checking,262.577659,0.000000,2020-01-04,False,True,False
1639,C000999,Ab5f7dac1,savings,1576.067421,0.000000,2019-05-21,False,False,True
1640,C000999,A261c8da8,credit_card,710.652416,31979.358719,2022-12-17,True,False,False
1641,C001000,Aeab65b63,checking,165.978184,0.000000,2019-04-02,False,True,False


In [44]:
# Aggregation of variables
aggregate_df = pd.DataFrame()
# Add each unique CustomerID as the index for aggregation
aggregate_df['CustomerID'] = accounts_df['CustomerID'].unique()
aggregate_df = aggregate_df.set_index('CustomerID')

# Add column for total balance across accounts
aggregate_df['TotalBalance'] = accounts_df.groupby('CustomerID')['Balance'].sum()
aggregate_df.head()

# Add column for Limit
aggregate_df['Limit'] = accounts_df.groupby('CustomerID')['Limit'].sum()

# Add column for number of account types
aggregate_df['NumCredit'] = accounts_df.groupby('CustomerID')['IsCredit'].sum()
aggregate_df['NumChecking'] = accounts_df.groupby('CustomerID')['IsChecking'].sum()
aggregate_df['NumSavings'] = accounts_df.groupby('CustomerID')['IsSavings'].sum()

# Change Limit back to nan for customers who have no credit accounts
aggregate_df.loc[(aggregate_df['NumCredit'] == 0), 'Limit'] = np.nan
aggregate_df

,TotalBalance,Limit,NumCredit,NumChecking,NumSavings
CustomerID,,,,,
C000001,535.150521,24081.773437,1,0,0
C000002,338.889874,8485.566470,1,1,0
C000003,5296.474584,194546.095136,1,2,0
C000004,1175.699220,NaN,0,0,1
C000005,300.342311,NaN,0,1,0
...,...,...,...,...,...
C000996,1036.431939,46639.437244,2,0,0
C000997,1026.324145,NaN,0,0,1
C000998,255.363386,11491.352383,1,0,0


In [45]:
aggregate_df.to_csv('agg_acc_all.csv')